<a href="https://colab.research.google.com/github/podschwadt/fhe_tutorial/blob/main/concrete_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building everything from scratch can be quite cumbersome. An alternative is writing our code as we used to and let a compiler handle the transformation to FHE. In this notebook we will do a small introduction to Concrete by Zama https://github.com/zama-ai/concrete

In [ ]:
!pip install concrete-python

We'll use the our preceptron from the previous notebook as again. It is able to distinguish between handwirtten 0s and 1s.

In [ ]:
import sklearn
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt


digits = datasets.load_digits()

zeroes = digits['images'][digits['target'] == 0]
ones = digits['images'][digits['target'] == 1]


x = np.concatenate((zeroes, ones))
y = np.concatenate((np.zeros(len(zeroes)), np.ones(len(ones))))
x,y = sklearn.utils.shuffle(x,y)

x_train, y_train = x[:300], y[:300]
x_test, y_test = x[300:], y[300:]

from sklearn.linear_model import Perceptron

classifier = Perceptron()
classifier.fit(x_train.reshape(len(x_train),-1), y_train)
classifier.score(x_test.reshape(len(x_test),-1), y_test)

Before we run the perceptron on encrypted data, let's look at how the compiler works on a very simple example. We start by importing the relevant package.

In [ ]:
from concrete import fhe

Then we define the function that we want to run on encyrpted data. We use a simple function that adds to values

In [ ]:
def add(x, y):
    return x + y

Next we create an instance of the compiler. Besides the function we want to compile we need to to tell the compiler what inputs are going to be `encrypted` and which will be in plaintext or `clear`.

In [ ]:
compiler = fhe.Compiler(add, {'x': 'encrypted', 'y': 'encrypted'})

The compiler needs to know what value ranges the function will encouter. We need to provide it with a list of example values.

In [ ]:
inputset = [(2, 3), (0, 0), (1, 6), (7, 7), (7, 1), (3, 2), (6, 1), (1, 7), (4, 5), (5, 4)]

We can then compile the function into an FHE circuit using the inputset

In [ ]:
circuit = compiler.compile(inputset)

The complier also determines what keys are required to run the circuit. We only need to create them.

In [ ]:
circuit.keygen()

Now we have everything to encrypt the data

In [ ]:
encrypted_x, encrypted_y = circuit.encrypt(2, 6)

We can now use the encrypted data to run the computation.

In [ ]:
encrypted_result = circuit.run(encrypted_x, encrypted_y)

Let's decrypt the result and see if it matches the plain computation

In [ ]:
result = circuit.decrypt(encrypted_result)
result == add(2, 6)

Let's get back to our preceptron example. First we'll reformat the data a little bit so we can work with more easily.

In [ ]:
# transform the data for easier use
x_train = x_train.reshape(len(x_train),-1).astype(int)
x_test = x_test.reshape(len(x_test),-1).astype(int)
weights = classifier.coef_.reshape(-1).astype(int)
bias = classifier.intercept_[0].astype(int)

Let's create function that does the preceptron inference. Recall that the perceptron function is $x \cdot w + b$ or spelled out $b + \sum x_iw_i$

In [ ]:
def perceptron_fhe(x, weights, bias):
  result = bias
  for x_, w in zip(x, weights):
    result += x_ * w
  return result

We can now compile the preceptron function. In this case only the input x is `encrypted`. The weights and bias will be in the `clear`.

Often finding a suitable input set can be tricky. In the case of  machine learning we are in a lucky postition. We can use the training data as our inputset.

In [ ]:
compiler = fhe.Compiler(perceptron_fhe, {'x': 'encrypted', 'weights': 'clear', 'bias': 'clear'})
inputset = [ (x, weights, bias) for x in x_train ]
circuit = compiler.compile(inputset)

Now we just need to encrypt the input and we can run the encrypted computation.

In [ ]:
print(f'Key generation...')
circuit.keygen()

print(f'Homomorphic evaluation...')
encrypted_x, _, _ = circuit.encrypt(x_test[0], weights, bias)
encrypted_result = circuit.run(encrypted_x, weights, bias)
result = circuit.decrypt(encrypted_result)


Is the output correct?

In [ ]:
classifier.decision_function(x_test[0].reshape(1,-1)) == result

Bonus Question:

Modify the code to meassure the time of the encrypted execution. Then change the function so that all inputs are encrytepd, including weights and biases.
What do you observe?

In [ ]:
# timing stuff in python
import time
start =  time.time()
# function to meassure
end = time.time()
print(f'Time in seconds: {end - start}')